In [1]:
from pathlib import Path
from transformers import BertTokenizer
from dataset import get_hypernyms_list_from_train, HypoDataset


data_path = Path('/home/hdd/data/hypernym/')
corpus_path = data_path / 'corpus.news_dataset-sample.token.txt'
hypo_index_path = data_path / 'index.full.news_dataset-sample.json'
train_set_path = data_path / 'train.cased.json'
candidates_path = data_path / 'candidates.cased.tsv'
wordnet_path = Path('/home/vimary/code-projects/dialog-2020-challenge/taxonomy-enrichment/data')

model_path = Path('/home/hdd/models/rubert_v2/rubert_cased_L-12_H-768_A-12_v2/')
tokenizer_vocab_path = model_path / 'vocab.txt'

In [2]:
tokenizer = BertTokenizer(tokenizer_vocab_path, do_lower_case=False)

In [3]:
# train_hype_list = get_hypernyms_list_from_train(train_set_path)

# ds = HypoDataset(tokenizer,
#                  corpus_path,
#                  hypo_index_path,
#                  train_set_path,
#                  train_hype_list)

# corpus = df.corpus
# index = ds.hypo_index[hypo]

In [4]:
# sample = ds.train_set[23]
# sample

In [5]:
# hypo = sample[0][0]
# hypo

In [6]:
from corpus_indexed import CorpusIndexed

corpus_indexed = CorpusIndexed(hypo_index_path)

index = corpus_indexed.idx
corpus = corpus_indexed.corpus

Loading index from /home/hdd/data/hypernym/index.full.news_dataset-sample.json.
Loading corpus from /home/hdd/data/hypernym/corpus.news_dataset-sample.token.txt.


### Explore predictions for specific hypo

In [7]:
hypo = 'эпилепсия'

In [8]:
hypo in index

True

In [10]:
count = 0
for corp_id, start_idx, end_idx in index[hypo]:
    count += 1
    text = corpus[corp_id]
    
    print(count, '\t', text.split()[start_idx: end_idx])
    print(text)

1 	 ['эпилепсии']
Как удалось выяснить американским учёным , что фрукты и мясо полезны при эпилепсии .
2 	 ['эпилепсии']
Учёные из США разработали рацион , уменьшающий риск приступов при смертельно опасной форме эпилепсии .
3 	 ['эпилепсией']
Как сообщает издание , врачи « прописали » больным эпилепсией продукты , содержащие мало углеводов и много жиров .
4 	 ['эпилепсия']
Мать , указав на младшую , пояснила , что у нее судороги и потребовала оформить сигнальный лист с диагнозом « судороги , эпилепсия » .
5 	 ['эпилепсия']
По данным прокуратуры , после одного из ДТП , в котором побывала девушка , ей поставили диагноз « эпилепсия » .
6 	 ['эпилепсии']
Как рассказал " Индустриалке " глава независимого профсоюза сотрудников " скорой помощи " Анатолий Сидоренко , в результате аварии одного из водителей доставили в больницу с приступом эпилепсии .
7 	 ['эпилепсии']
Медики вкололи Роналду гарденал - препарат , который помогает побороть приступ эпилепсии , но губителен для сердца .
8 	 ['эпил

In [11]:
from embedder import get_word_embeddings
from transformers import BertConfig, BertTokenizer, BertModel
import torch

In [12]:
config = BertConfig.from_pretrained(model_path / 'bert_config.json')
tokenizer = BertTokenizer.from_pretrained(model_path, do_lower_case=False)
model = BertModel.from_pretrained(str(model_path / 'ptrubert.pt'), config=config)

emb_mat = model.embeddings.word_embeddings.weight

In [13]:
from utils import get_wordnet_synsets, enrich_with_wordnet_relations, synsets2senses


wordnet_synsets = get_wordnet_synsets(wordnet_path.glob('synsets.N*'))
# wordnet_synsets = get_wordnet_synsets(wordnet_path.glob('synsets.*'))
# enrich_with_wordnet_relations(wordnet_synsets, wordnet_path.glob('synset_relations.*'))

Parsing /home/vimary/code-projects/dialog-2020-challenge/taxonomy-enrichment/data/synsets.N.xml.


In [36]:
LEVEL = 'sense'

if LEVEL == 'synset':
    # embedding all synsets (list of phrases)
    hype_embs, hype_list = [], []
    for s_id, synset in wordnet_synsets.items():
        if s_id[-1] != 'N':
            print(f'skipping {s_id}')
        hype_list.append(synset['ruthes_name'].lower())
        senses = [sense['content'].lower() for sense in synset['senses']]
        hype_embs.append(get_word_embeddings(senses, emb_mat, tokenizer=tokenizer).mean(dim=0))
    hype_embs = torch.stack(hype_embs, dim=0)
elif LEVEL == 'sense':
    # embedding all senses (phrases)
    hype_list = sorted(set(synset['ruthes_name'].lower()
                           for s_id, synset in wordnet_synsets.items()
                           if s_id[-1] == 'N'))
    hype_embs = get_word_embeddings(hype_list, emb_mat, tokenizer=tokenizer)
print(f"Hypernym embeddings are of shape {hype_embs.shape}")

Hypernym embeddings are of shape torch.Size([29296, 768])


In [53]:
NUM_CONTEXTS = 32


token_idxs, hypo_masks = [], []
max_len = 0

for i, (corp_id, start_idx, end_idx) in enumerate(index[hypo]):
    text = corpus[corp_id]
    
    subtokens, mask = ['[CLS]'], [0.0]
    for n, token in enumerate(text.split()):
        current_subtokens = tokenizer.tokenize(token)
        subtokens.extend(current_subtokens)
        mask_val = 1.0 if n in range(start_idx, end_idx) else 0.0
        mask.extend([mask_val] * len(current_subtokens))
    subtokens.append('[SEP]')
    mask.append(0.0)
    
    token_idxs.append(tokenizer.convert_tokens_to_ids(subtokens))
    hypo_masks.append(mask)
    max_len = max_len if max_len > len(subtokens) else len(subtokens)
    if i >= NUM_CONTEXTS - 1:
        break
    
# pad to max_len
attn_masks = [[1.0] * len(idxs) + [0.0] * (max_len - len(idxs)) for idxs in token_idxs]
token_idxs = [idxs + [0] * (max_len - len(idxs)) for idxs in token_idxs]
hypo_masks = [mask + [0.0] * (max_len - len(mask)) for mask in hypo_masks]
    
# h: [batch_size, seq_len, hidden_size]
h, v = model(torch.tensor(token_idxs), attention_mask=torch.tensor(attn_masks))
# hypo_mask_t: [batch_size, seq_len]
hypo_mask_t = torch.tensor(hypo_masks)
# r: [batch_size, seq_len, hidden_size]
r = h * hypo_mask_t.unsqueeze(2)
# r: [batch_size, hidden_size]
r = r.sum(dim=1) / hypo_mask_t.sum(dim=1, keepdim=True)
# # r: [hidden_size]
# r = r.mean(dim=0)

In [55]:
METRIC = 'product'

# scores: [batch_size, vocab_size]
if METRIC == 'cosine':
    # hype_embs_norm: [vocab_size, hidden_size]
    hype_embs_norm = hype_embs / hype_embs.norm(dim=1, keepdim=True)
    scores = r @ hype_embs_norm.T / r.norm()
elif METRIC == 'product':
    scores = r @ hype_embs.T
    
scores = torch.log_softmax(scores, dim=1)
# scores: [vocab_size]
scores = scores.mean(dim=0)

In [56]:
vals, indices = torch.topk(scores, 50)

In [57]:
r.norm()

tensor(83.6648, grad_fn=<NormBackward0>)

In [58]:
idx = 6154

hype_embs[idx].norm()

tensor(0.4623, grad_fn=<NormBackward0>)

In [59]:
(hype_embs[idx] * r).sum()

tensor(-14.0973, grad_fn=<SumBackward0>)

In [60]:
'эпилепсия' in hype_list

False

In [61]:
for score, i in zip(vals.tolist(), indices.tolist()):
    print(f'{hype_list[i]:25} {score:.2} {i:10}')

тесть                     -9.4      25261
шизофрения                -9.4      28361
сердечный приступ         -9.4      22619
нерв                      -9.4      14144
томь                      -9.4      25463
кольт                     -9.4       9738
паралич                   -9.5      16479
бор                       -9.5       2076
калла                     -9.5       8774
перила                    -9.5      17115
бидон                     -9.5       1721
разорваться на части      -9.5      20736
корье                     -9.5      10265
экстази                   -9.5      28772
человек                   -9.5      27929
корь                      -9.5      10264
прикарпатье               -9.6      19292
шкив                      -9.6      28405
время года                -9.6       3701
секция организации        -9.6      22494
больной человек           -9.6       2051
драматический театр       -9.6       6369
склероз                   -9.6      22907
лошадь                    -9.6    